In [736]:
from utils import *

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

np.set_printoptions(formatter={'float': '{: 0.2f}'.format})

In [737]:
# Bunny point cloud
# path_to_bunny = "D:\\Skole\\Semester 10\\Prosjektoppgave\\Data\\bunny\\reconstruction\\bun_zipper.ply"
path_to_bunny = "D:\\Runar\Documents\\1_Studier\\Semester_10\\Prosjektoppgave\\22v-TPK4560-Prosjektoppgave-Pose_estimation\\Data\\bunny" +\
    "\\reconstruction\\bun_zipper_res2.ply"
    # "\\data\\bun000.ply"
bunny_ply = o3d.io.read_point_cloud(path_to_bunny)
bunny = np.asarray(bunny_ply.points)
bunny_true = np.copy(bunny)
n = bunny.shape[0]

In [738]:
# Correct rotation
rot = np.array([np.pi/3,
              np.pi/3,
              np.pi/3
            ])
# Noise and outliers
# noise_1 = np.array([np.pi/3 + np.random.normal(0,0.05),
#                     np.pi/3 + np.random.normal(0,0.05),
#                     np.pi/3 + np.random.normal(0,0.05)
#                     ])
outlier = np.array([np.pi * np.random.uniform(-1,1),
                    np.pi * np.random.uniform(-1,1),
                    np.pi * np.random.uniform(-1,1)
                    ])
      

In [739]:
bunny = bunny @ expso3(rot)

In [740]:
# Generating translated cloud with inaccuracies
for i, p in enumerate(bunny):
    break
    # break
    if i % 4 == 0:
        bunny[i] = expso3(rot) @ p
        continue
    else:
        bunny[i] = expso3(rot) @ p 
        # + np.random.uniform(-.1,.2, 3)
        # outlier = np.array([np.pi*np.random.uniform(-1,1),
        #             np.pi*np.random.uniform(-1,1),
        #             np.pi*np.random.uniform(-1,1)
        #             ])
        continue

In [741]:
# GNC Initialization
# Initial rotation guess
R0 = np.identity(3)

# Initial loss function data
r = np.zeros(n)
rtest = np.zeros(n)
for i in range(n):
    r[i] = np.linalg.norm(bunny_true[i] - R0 @ bunny[i])
    rtest[i] = np.linalg.norm(expso3(rot) @ bunny_true[i] - R0 @ bunny_true[i])
r0_max = np.max(r)

# gnc
eps = 0.011 
mu_update_factor = 1.4
max_iterations = 1000
w = np.ones(n)
mu = eps**2 / (2*r0_max**2 - eps**2)

In [742]:
last_iter = []
R_iter = [np.sum(rtest)]
iterations = 0
for i in range(max_iterations):
    iterations += 1
    last_iter.append(np.sum(w))
    # Weighted Procrustes
    H = bunny.T @ np.diag(w) @ bunny_true
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ np.diag([1,1,np.linalg.det( Vt.T @ U.T)]) @ U.T

    # Loss function
    for j in range(n):
        r[j] = np.linalg.norm(bunny_true[j] - R @ bunny[j])
        w[j] = w_from_r(r[j], eps, mu)
        rtest[j] = np.linalg.norm(expso3(rot) @ bunny_true[j] - R @ bunny_true[j])



    R_iter.append(np.sum(rtest))

    mu = mu_update_factor * mu

    if i >= 5:
        if np.sum(w) == last_iter[i]:
            break

In [743]:
print(R)
print()
print(expso3(rot))
print()
print(expso3(rot) - R)

[[ 0.17 -0.15  0.97]
 [ 0.97  0.17 -0.15]
 [-0.15  0.97  0.17]]

[[ 0.17 -0.15  0.97]
 [ 0.97  0.17 -0.15]
 [-0.15  0.97  0.17]]

[[-0.00  0.00  0.00]
 [-0.00  0.00  0.00]
 [-0.00 -0.00 -0.00]]


In [744]:
inliers = []
outliers = []

for i, n in enumerate(w):
    if w[i] == 1.0:
        inliers.append(n)
    else:
        outliers.append(n)

percentage = (1-len(inliers)/(len(outliers)+len(inliers)))*100

print("Inliers:\t", len(inliers), "\nOutliers:\t", len(outliers),\
    "\nPercentage:\t",percentage, "%\n\nIterations:\t", iterations)

Inliers:	 8171 
Outliers:	 0 
Percentage:	 0.0 %

Iterations:	 6


In [745]:
geometry_list = []

In [746]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(bunny)
# o3d.visualization.draw_geometries([pcd], width=1600, height=900)
# geometry_list.append(pcd)

In [747]:
color1 = np.array(([1.0, 0.0, 0.0]))
color2 = np.array(([0.0, 1.0, 0.0]))
color3 = np.array(([0.0, 0.0, 1.0]))
pcdtest1 = o3d.geometry.PointCloud()
pcdtest2 = o3d.geometry.PointCloud()
pcdtest3 = o3d.geometry.PointCloud()

In [748]:
bunnytest = bunny_true.copy()
test1 = bunnytest @ expso3(rot)
pcdtest1.points = o3d.utility.Vector3dVector(test1)
test2 = bunnytest @ R + np.ones((bunnytest.shape[0],3))*0.001
pcdtest2.points = o3d.utility.Vector3dVector(test2)
test3 = np.zeros((bunnytest.shape[0],3))
for i, x in enumerate(bunnytest):
    test3[i,:] = w[i] * x
pcdtest3.points = o3d.utility.Vector3dVector(test3)
# testfinal = np.concatenate((bunnytest, test1, test2, test3), axis=0)

In [749]:
pcdtest1.paint_uniform_color(color1)
pcdtest2.paint_uniform_color(color2)
pcdtest3.paint_uniform_color(color3)

geometry_list.append(pcdtest1)
geometry_list.append(pcdtest2)
geometry_list.append(pcdtest3)

In [750]:
# pcd1 = o3d.geometry.PointCloud()
# pcd1.points = o3d.utility.Vector3dVector(testfinal)
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1, origin=[0, 0, 0])
geometry_list.append(mesh_frame)
o3d.visualization.draw_geometries(geometry_list, width=1600, height=900)

In [751]:
print(last_iter)

[8171.0, 8171.0, 8171.0, 8171.0, 8171.0, 8171.0]


In [752]:
for i, n in enumerate(R_iter):
    print(n)

1224.2414191210391
9.051062138547187e-13
9.051062138547187e-13
9.051062138547187e-13
9.051062138547187e-13
9.051062138547187e-13
9.051062138547187e-13
